In [1]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 27.0 MB/s 


In [2]:
import numpy as np
import pandas as pd
import random
from datetime import timedelta
import numpy as np 
from datetime import datetime
import xlsxwriter

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Project files/data.csv')
df.head()

,Fraud_id,Account_no.,Account opening date,Transaction date,Account type,Fraud type,Transaction date new,Transaction_Amount
0,7726,98079,2021-09-27 19:23:36,2021-04-04 14:43:17,Current account,identity theft fraud,2021-09-28 19:23:36,48569
1,1767,15627,2021-07-05 08:02:11,2021-04-23 22:30:41,Current account,identity theft fraud,2021-07-06 08:02:11,3297
2,3724,82003,2021-08-29 05:10:57,2021-12-30 09:00:22,Savings account,identity theft fraud,2021-12-30 09:00:22,97107
3,9853,12340,2021-09-16 10:25:26,2021-12-08 23:20:52,Savings account,identity theft fraud,2021-12-08 23:20:52,76499
4,6223,34648,2021-08-11 05:38:34,2021-06-25 23:18:24,Savings account,Cyber fraud,2021-08-12 05:38:34,92150


In [9]:
df['Account opening date'] = pd.to_datetime(df['Account opening date'])

In [10]:
df['Transaction date new'] = pd.to_datetime(df['Transaction date new'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Fraud_id              10000 non-null  int64         
 1   Account_no.           10000 non-null  int64         
 2   Account opening date  10000 non-null  datetime64[ns]
 3   Transaction date      10000 non-null  object        
 4   Account type          10000 non-null  object        
 5   Fraud type            10000 non-null  object        
 6   Transaction date new  10000 non-null  datetime64[ns]
 7   Transaction_Amount    10000 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 625.1+ KB


In [ ]:
# What’s is days difference between account opening date and transaction date?

# Try to answer the following questions in your report. 

# 1. What’s is days difference between account opening date and transaction date?(excluding weekends)

# 2. What’s is the transaction break up of 1 month from account opening date.

# 3. What is the monthly break up for transaction?

# 4. What is the account type break up? 


# You can add your few more questions from your side and create report

In [12]:
df['Diff_Without_Weekend'] = df.apply(lambda x : np.busday_count( x['Account opening date'].date() , x['Transaction date new'].date()), axis = 1)

In [13]:
df['First_30_days'] = np.where(df['Diff_Without_Weekend'] <= 30 , 'First_30days_excluding_weekends' , 'greaterthan_30days_excluding_weekends')

In [14]:
df.sort_values(by = 'Transaction date', inplace = True)

In [15]:
fraud_amt = df.groupby(['First_30_days','Fraud type','Account type'])['Transaction_Amount'].agg(['count','sum','mean']).reset_index()

In [16]:
df['Transaction date new'] = df['Transaction date new'].apply(lambda  x:  x.date())

df['Transaction_Month'] = df['Transaction date new'].apply(lambda x : x.replace(day=1))

In [17]:
tnx_amt_df = df.groupby(['Transaction_Month','Fraud type'])['Transaction_Amount'].agg(['count','sum','mean']).reset_index()

In [18]:
sum_tnx_amt= tnx_amt_df.pivot(index='Fraud type', columns='Transaction_Month', values='sum')
sum_tnx_amt.columns = [i.strftime('%B-%Y') for i in tnx_amt_df['Transaction_Month'].unique()]
sum_tnx_amt = sum_tnx_amt.reset_index()

In [19]:
count_tnx_amt= tnx_amt_df.pivot(index='Fraud type', columns='Transaction_Month', values='count')
count_tnx_amt.columns = [i.strftime('%B-%Y') for i in tnx_amt_df['Transaction_Month'].unique()]
count_tnx_amt = count_tnx_amt.reset_index()

In [20]:
mean_tnx_amt= tnx_amt_df.pivot(index='Fraud type', columns='Transaction_Month', values='mean')
mean_tnx_amt.columns = [i.strftime('%B-%Y') for i in tnx_amt_df['Transaction_Month'].unique()]
mean_tnx_amt = mean_tnx_amt.reset_index()

In [21]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('Report.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('Monthly Report')
writer.sheets['Monthly Report'] = worksheet
worksheet=workbook.add_worksheet('First_30_Days_Fraud_Report')
writer.sheets['First_30_Days_Fraud_Report'] = worksheet
pd.DataFrame(columns = ['Count Of Fraud Transaction']).to_excel(writer,sheet_name='Monthly Report',startrow= 1 , startcol=0)
count_tnx_amt.to_excel(writer,sheet_name='Monthly Report',startrow=2, startcol=0)
pd.DataFrame(columns = ['Sum Of Fraud Transaction Amount']).to_excel(writer,sheet_name='Monthly Report',startrow= 7 , startcol=0)
sum_tnx_amt.to_excel(writer,sheet_name='Monthly Report',startrow=8 , startcol=0)  
pd.DataFrame(columns = ['Mean Of Fraud Transaction Amount']).to_excel(writer,sheet_name='Monthly Report',startrow= 13 , startcol=0)
mean_tnx_amt.to_excel(writer,sheet_name='Monthly Report',startrow= 14 , startcol=0)
pd.DataFrame(columns = ['First_30_Days_Excluding_Weekend_From_Account_Opening_Fraud_Report']).to_excel(writer,sheet_name='First_30_Days_Fraud_Report',startrow= 1 , startcol=0)
fraud_amt.to_excel(writer,sheet_name='First_30_Days_Fraud_Report',startrow= 2 , startcol=0)

In [22]:
writer.close()

In [23]:
from google.colab import files
files.download('Report.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>